In [1]:
from pathlib import Path
import pandas as pd
import gc
from sklearn.preprocessing import LabelEncoder
import pickle
import bz2

Download https://drive.google.com/drive/folders/1BZFez3OFs7QMzeDUz-8E0SgnRy227pnG

#### Task

    1. Prove that buying discounted products depends on the average order sum. (It is more probable that the discounted product will be bought if a bunch of money is going to spend)
    2. Prove that the frequency of shopping grows around the wage taking days (1-6 days of each month, 20 -26 days of each month)

Define such cohort existence:

    * Frequent buyers - The users that are shopping frequently(daily, weekly, monthly) 
    * Average purchase sum - The customers that usually spend the same amount of money 
    * Usual cart - The customers grouped by the product groups

Create a pull request by using a format hw2;lastname;firstname

In [3]:
data_path = Path('./data/').resolve()
!ls {data_path}

09_groups_cleaned.csv			10_groups.csv	   outliers.pkl.bz2
09_groups_cleaned_multiindex_small.csv	11_groups.csv	   product_groups.csv
09_groups_cleaned_small.csv		bad_ids.csv
09_groups.csv				inno_stats.tar.gz


In [4]:
g_9 = pd.read_csv(data_path / '09_groups.csv', 
                  usecols=['date', 'id_doc', 'id_card', 'sum', 'is_green', 'id_tov'], 
                  dtype=dict(is_green=bool))
g_9.date = g_9.date.str[8:10].apply(int)
g_9.head()

,date,id_doc,id_card,id_tov,sum,is_green
0,1,1283228,1538855,52,108.00,False
1,1,8873113,267307,52,108.00,False
2,1,12712899,610220,52,107.95,False
3,1,21535283,441497,52,108.00,False
4,1,642341,1065358,61,88.00,False


The raw dataset consists of 9 different columns:

* date - is the timestamp when the purchase was done
* id_doc - receipt's id
* id_order - is the order's id. Order's id is equal to 0 when the purchase was done offline, otherwise online.
* id_card - is the unique id of customer.
* id_tov - is the unique id of product's id
* id_kontr - distributor's id
* quantity - the amount of the same product in receipt which was purchased
* sum - price of the product
* is_green - shows that product is under discount or not

In [5]:
g_9.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42100858 entries, 0 to 42100857
Data columns (total 6 columns):
 #   Column    Dtype  
---  ------    -----  
 0   date      int64  
 1   id_doc    int64  
 2   id_card   int64  
 3   id_tov    int64  
 4   sum       float64
 5   is_green  bool   
dtypes: bool(1), float64(1), int64(4)
memory usage: 1.6 GB


Remove outliers:

In [6]:
outliers_df = pd.read_csv('./data/bad_ids.csv')
outliers_df.head()

,Unnamed: 0,id_card
0,0,806096
1,1,1438102
2,2,72344
3,3,427981
4,4,1528414


In [7]:
outliers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24707 entries, 0 to 24706
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   Unnamed: 0  24707 non-null  int64
 1   id_card     24707 non-null  int64
dtypes: int64(2)
memory usage: 386.2 KB


In [8]:
cond = g_9['id_card'].isin(outliers_df['id_card'])
g_9.drop(g_9[cond].index, inplace = True)

In [10]:
g_9.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38466003 entries, 0 to 42100857
Data columns (total 6 columns):
 #   Column    Dtype  
---  ------    -----  
 0   date      int64  
 1   id_doc    int64  
 2   id_card   int64  
 3   id_tov    int64  
 4   sum       float64
 5   is_green  bool   
dtypes: bool(1), float64(1), int64(4)
memory usage: 1.8 GB


In [11]:
g_9.to_csv('./data/09_groups_cleaned_small.csv')

In [12]:
g_9.set_index(['id_card', 'id_doc', 'date'], inplace=True)

In [13]:
g_9.sort_index(inplace=True)

In [14]:
g_9

id_tov    sum  is_green
id_card id_doc   date                         
0       60540    10       647    5.0     False
                 10     21193   79.0     False
                 10     31712  167.0     False
        4304074  23     15774   87.0     False
                 23     19081   24.0     False
...                       ...    ...       ...
1944074 15011738 14       173  104.0     False
                 14     23760   88.2      True
                 14     37333   52.8      True
        18208481 14     17823   46.0     False
                 14     22984  100.0     False

[38466003 rows x 3 columns]

As seen from the table above, most of the orders consist of multiple products. Moreover, for simplicity let us assume the following: if a person bought something at the same day but at different time, it is still the same purchase. Thus we can neglect the timestamps from the original data.

Intuitively: if we get back to a store, we most probably forgot to buy something we intended to. So, if it was not for our memory, we would have bought the item during the first visit.

In [4]:
g_9 = pd.read_csv('./data/09_groups_cleaned_small.csv')
g_9

,Unnamed: 0,date,id_doc,id_card,id_tov,sum,is_green
0,0,1,1283228,1538855,52,108.00,False
1,1,1,8873113,267307,52,108.00,False
2,2,1,12712899,610220,52,107.95,False
3,3,1,21535283,441497,52,108.00,False
4,4,1,642341,1065358,61,88.00,False
...,...,...,...,...,...,...,...
38465998,42100853,30,4850561,1739990,25292,130.00,False
38465999,42100854,30,10015419,1761351,30740,120.00,False
38466000,42100855,30,22816791,1432778,30741,73.00,False
38466001,42100856,30,1648929,484390,34835,78.00,False


## Task 1

Prove that buying discounted products depends on the average order sum. (It is more probable that the discounted product will be bought if bunch of money is going to be spend)

Let us formulate the hypotheses (here we will formulate them in terms of correlation because we should not assume causation):
* H_0: there is a positive correlation between the number of discounted products (in percents to overall number of products in an order) and the total sum spent on the order.
* H_1: there is no such correlation or the correlation is negative

In [5]:
from scipy.stats import pearsonr, spearmanr
import numpy as np

In [8]:
sum_per_order = g_9[['sum', 'id_doc']].groupby("id_doc").sum()['sum'].to_list()

n_discounts = g_9[['id_doc', 'is_green']].groupby("id_doc")['is_green'].sum().to_list()
n_orders = g_9[['sum', 'id_doc']].groupby("id_doc").count()['sum'].to_list()

discount_percent = (np.array(n_discounts)/np.array(n_orders))

In [9]:
pearsonr(sum_per_order, discount_percent)

(-0.11211405758848686, 0.0)

The results show a negative pearson correlation coefficient of approximately -0.11 with a p-value close to 0.0: Reject the null hypothesis

## Task 2
Prove that the frequency of shopping grows around the wage taking days (1-6 days of each month, 20 -26 days of each month)

* H_0: frequency of shopping is the same no matter if it is a wage taking day or not
* H_1: frequency of shopping is different on wage taking days

In [3]:
wage_days = g_9[(g_9.date <= 6)  | ((g_9.date >= 20) & (g_9.date <= 26))]
wage_days

,Unnamed: 0,date,id_doc,id_card,id_tov,sum,is_green
0,0,1,1283228,1538855,52,108.00,False
1,1,1,8873113,267307,52,108.00,False
2,2,1,12712899,610220,52,107.95,False
3,3,1,21535283,441497,52,108.00,False
4,4,1,642341,1065358,61,88.00,False
...,...,...,...,...,...,...,...
33003151,36132115,26,22838804,1581217,38375,193.50,False
33003152,36132116,26,20090608,256794,103,113.60,False
33003153,36132117,26,12316511,1347234,22985,100.00,False
33003154,36132118,26,7568618,1347234,26512,94.50,False


In [4]:
wage_days_docs = wage_days['id_doc'].values
other_days = g_9[~g_9['id_doc'].isin(wage_days_docs)]
other_days

,Unnamed: 0,date,id_doc,id_card,id_tov,sum,is_green
7324461,8038412,7,3391578,1212752,52,216.00,False
7324462,8038414,7,5805911,332802,52,215.78,False
7324463,8038415,7,8230006,267237,52,108.00,False
7324464,8038416,7,9832258,1408399,52,108.00,False
7324465,8038417,7,19514208,563157,52,174.00,False
...,...,...,...,...,...,...,...
38465998,42100853,30,4850561,1739990,25292,130.00,False
38465999,42100854,30,10015419,1761351,30740,120.00,False
38466000,42100855,30,22816791,1432778,30741,73.00,False
38466001,42100856,30,1648929,484390,34835,78.00,False


In [5]:
# calculate the number of different product items a person buys when receives wage (not a receipt)
n_products_wage = wage_days[['date', 'id_doc', 'id_card']].groupby(["id_doc", 'id_card'], as_index=False).count()  
n_products_wage.rename(columns={'date': 'n_items_bought'}, inplace=True)
n_products_wage

,id_doc,id_card,n_items_bought
0,2,1425208,6
1,4,1438933,17
2,5,293299,2
3,19,948401,4
4,26,1530864,1
...,...,...,...
2973040,23417844,619644,2
2973041,23417845,508562,3
2973042,23417859,781677,3
2973043,23417862,480091,5


In [6]:
freq_wage = n_products_wage.groupby(['id_card'], as_index=False)['n_items_bought'].sum() / 12
freq_wage

,id_card,n_items_bought
0,0.000000,0.666667
1,0.083333,0.083333
2,0.166667,1.000000
3,0.250000,1.916667
4,0.333333,2.166667
...,...,...
999888,161007.666667,0.333333
999889,161074.416667,0.500000
999890,161175.250000,2.000000
999891,161618.083333,2.166667


In [8]:
# calculate the number of different product items a person buys in other days 
n_products_other_days = other_days[['date', 'id_doc', 'id_card']].groupby(["id_doc", 'id_card'], as_index=False).count()  
n_products_other_days.rename(columns={'date': 'n_items_bought'}, inplace=True)
n_products_other_days

,id_doc,id_card,n_items_bought
0,16,715970,7
1,18,74800,7
2,27,200044,7
3,31,788119,2
4,32,1459804,3
...,...,...,...
3931775,23417837,365727,3
3931776,23417853,36431,20
3931777,23417857,288143,2
3931778,23417864,1392399,5


In [9]:
freq_other_days = n_products_other_days.groupby(['id_card'], as_index=False)['n_items_bought'].sum() / (30-12)
freq_other_days

,id_card,n_items_bought
0,0.000000,2.000000
1,0.166667,0.611111
2,0.222222,0.555556
3,0.444444,0.277778
4,0.555556,3.777778
...,...,...
1097903,107384.666667,0.166667
1097904,107389.611111,0.055556
1097905,107410.277778,1.944444
1097906,107745.388889,2.111111


In [10]:
freq = freq_wage[['id_card', 'n_items_bought']].merge(freq_other_days[['id_card', 'n_items_bought']], 
                                                      on='id_card', how='outer', suffixes=('_wage', '_other'))
freq.fillna(0, inplace=True)
freq

,id_card,n_items_bought_wage,n_items_bought_other
0,0.000000,0.666667,2.000000
1,0.083333,0.083333,0.000000
2,0.166667,1.000000,0.611111
3,0.250000,1.916667,0.000000
4,0.333333,2.166667,0.000000
...,...,...,...
1897368,107384.666667,0.000000,0.166667
1897369,107389.611111,0.000000,0.055556
1897370,107410.277778,0.000000,1.944444
1897371,107745.388889,0.000000,2.111111


In [11]:
wage_freq_mean = freq['n_items_bought_wage'].mean()
other_days_freq_mean = freq['n_items_bought_other'].mean()
wage_freq_mean, other_days_freq_mean

(0.7215320428121756, 0.6452728178498494)

In [12]:
from scipy.stats import ttest_ind
ttest_ind(freq['n_items_bought_wage'], freq['n_items_bought_other'])

Ttest_indResult(statistic=58.952775690840056, pvalue=0.0)

The p-value is really small, so we reject the null hypothesis of equal averages.
This means that the frequency of purchase (the number of purchased items) on wage days considered for each customer differs from the customer's intensity od purchase on other days.

## Task 3
Define such cohort existence:
1. Frequent buyers - The users that are shopping frequently(daily, weekly, monthly) 
2. Average purchase sum - The customers that usually spend the same amount of money 
4. Usual cart - The customers grouped by the product groups

References
- https://towardsdatascience.com/a-step-by-step-introduction-to-cohort-analysis-in-python-a2cbbd8460ea

In [2]:
from pathlib import Path
import pandas as pd
# import gc
# from sklearn.preprocessing import LabelEncoder
# import pickle
# import bz2

Frequent buyers:

In [3]:
freq_buyers = pd.read_csv('./data/09_groups_cleaned_small.csv',
                 usecols=['id_doc', 'id_card'])
freq_buyers = freq_buyers.groupby('id_card', as_index=False).count()
freq_buyers.rename(cols={'id_doc': 'frequency'})

In [4]:
freq_buyers

,id_card,id_doc
0,0,44
1,1,1
2,2,12
3,3,34
4,4,36
...,...,...
1333953,1933013,1
1333954,1933385,35
1333955,1934103,24
1333956,1939417,64


Average purchase sum:

In [5]:
avg_purchase = pd.read_csv('./data/09_groups_cleaned_small.csv',
                 usecols=['id_card', 'sum'])
avg_purchase = avg_purchase.groupby('id_card', as_index=False).mean()

In [6]:
avg_purchase

,id_card,sum
0,0,81.981818
1,1,147.000000
2,2,123.750000
3,3,129.217059
4,4,108.475556
...,...,...
1333953,1933013,165.000000
1333954,1933385,154.236286
1333955,1934103,124.250417
1333956,1939417,96.151719


Usual cart:

In [7]:
p_g = pd.read_csv('./data/product_groups.csv', delimiter=';', encoding='windows-1251')
p_g.head()

,id_tov,id_group,name_group
0,52,10004,Кисломолочные продукты
1,75,10004,Кисломолочные продукты
2,77,10004,Кисломолочные продукты
3,143,10004,Кисломолочные продукты
4,151,10004,Кисломолочные продукты


In [8]:
p_g.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5650 entries, 0 to 5649
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id_tov      5650 non-null   int64 
 1   id_group    5650 non-null   int64 
 2   name_group  5650 non-null   object
dtypes: int64(2), object(1)
memory usage: 132.5+ KB


In [15]:
tov_groups = pd.read_csv('./data/09_groups_cleaned_small.csv',
                 usecols=['id_card', 'id_group'])

tov_per_customer = tov_groups.groupby('id_group', as_index=False).count()